# Inverse Optimization for DFS
_Applying Ghobadi and Mahmoudzadeh 2021_

In [ ]:
using JuMP
using Gurobi
using LinearAlgebra

## Structures and parameters

In [2]:
struct ForwardProblemParams
    n_paths::Int
    n_commodities::Int

    capacities::Vector{Number}
    demands::Vector

    design_costs::Vector
    flow_costs::Matrix

    enabled_flows::Matrix{Bool}

    function ForwardProblemParams(; n_paths, n_commodities, capacities, demands, design_costs, flow_costs, enabled_flows=nothing)
        shape = (n_paths, n_commodities)
        
        if isnothing(enabled_flows) 
            enabled_flows = ones(Bool, shape)
        elseif size(enabled_flows) != shape
            error("Invalid shape $(size(enabled_flows)) for `disabled_flows`, should be $(shape)")
        end
        
        new(n_paths, n_commodities, capacities, demands, design_costs, flow_costs, enabled_flows)
    end
end

In [3]:
function forward_example_params()::ForwardProblemParams
    enabled_flows = ones(Bool, (2, 2))
    enabled_flows[1, 2] = false

    return ForwardProblemParams(
        n_paths=2, 
        n_commodities=2,
        capacities=[100, 100],
        demands=[10, 6],
        design_costs=[100, 10],
        flow_costs=[10 10 ; 100 100],
        enabled_flows=enabled_flows
    )
end

forward_example_params (generic function with 1 method)

## Forward problem

In [19]:
function create_forward_problem(params::ForwardProblemParams)::Model
    model = Model(Gurobi.Optimizer)

    @variable(model, z[1:params.n_paths], Bin)
    @variable(model, x[1:params.n_paths, 1:params.n_commodities] >= 0)

    @objective(model, Min, params.design_costs' * z + sum(params.flow_costs .* x))

    @constraint(model, [k = 1:params.n_commodities], sum(x[:, k]) .== params.demands[k])
    @constraint(model, [i = 1:params.n_paths], sum(x[i, :]) <= params.capacities[i] * z[i])
    @constraint(model, [i = 1:params.n_paths, k = 1:params.n_paths; !params.enabled_flows[i, k]], x[i, k] .== 0)

    return model
end

function solve_forward_problem!(model::Model)
    optimize!(model)

    x_sol = value.(model[:x])
    z_sol = value.(model[:z])

    return x_sol, z_sol
end

solve_forward_problem! (generic function with 1 method)

In [20]:
forward_model = create_forward_problem(forward_example_params())
x_sol, z_sol = solve_forward_problem!(forward_model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-23
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 6 columns and 11 nonzeros
Model fingerprint: 0xa664e8a1
Variable types: 4 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 1e+01]
Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 810 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.100000000000e+02, best bound 8.100000000000e+02, gap 0.0000%

User-callback calls 21

([10.0 0.0; 0.0 6.0], [1.0, 1.0])

In [21]:
println(x_sol)
println(z_sol)

[10.0 0.0; 0.0 6.0]
[1.0, 1.0]


## Inverse problem (single point)

In [39]:
function create_solution_vector(x_sol::Matrix, z_sol::Vector)
    flat_xs = reshape(x_sol, (length(x_sol), 1))

    return vec(vcat(flat_xs, z_sol))
end

function create_A(n_paths::Integer, n_commodities::Integer, n_variables::Integer, enabled_flows::Matrix{Bool})
    A_pos = zeros(Number, (n_commodities, n_variables))
    for k in 1:n_commodities
        shift_amount = (k - 1) * n_paths
        shifted_range = (1:n_paths) .+ shift_amount
        A_pos[k, shifted_range] .= Int.(params.enabled_flows[:,k])
    end
    A = vcat(A_pos, -A_pos)

    return A
end

function create_Gh(n_paths::Integer, n_commodities::Integer, n_variables::Integer, enabled_flows::Matrix{Bool})
    n_flows = n_paths * n_commodities

    G_paths = zeros(Number, (n_paths, n_variables))
    for p in 1:n_paths
        G_paths[p, p:n_commodities:n_commodities*n_paths] .= Int.(params.enabled_flows[p, :])
        G_paths[p, n_flows + p] = params.capacities[p]
    end
    G_nonneg = diagm(ones(n_variables))
    G_binary = hcat(zeros((n_paths, n_flows)), diagm(ones(n_paths)))
    
    G = vcat(.-G_paths, G_nonneg, G_binary)
    h = zeros(size(G)[1])

    return G,h
end

function create_AGh(params::ForwardProblemParams)
    n_paths, n_commodities = params.n_paths, params.n_commodities
    n_flows = n_paths * n_commodities
    n_variables = n_flows + n_paths

    A = create_A(n_paths, n_commodities, n_variables, params.enabled_flows)
    G,h = create_Gh(n_paths, n_commodities, n_variables, params.enabled_flows)

    return A, G, h
end

function add_half_space_constraint(G::Matrix, h::Vector, sol_opt::Vector, params::ForwardProblemParams)
    flat_flow_costs = reshape(params.flow_costs, (length(params.flow_costs), 1))
    full_costs = vcat(flat_flow_costs, params.design_costs)

    optimal_cost = full_costs' * sol_opt

    G_hs = vcat(G, full_costs')
    h_hs = vcat(h, optimal_cost)

    return G_hs, h_hs
end

add_half_space_constraint (generic function with 2 methods)

In [43]:

function create_b_variables!(model::Model, n_commodities::Integer)
    @variable(model, b[1:(2 * n_commodities)])
    @constraint(model, [i = 1:n_commodities], b[i] == -b[n_commodities + i])
    
    return b
end

function add_inverse_constraints!(model::Model, sol::Vector, A::Matrix, b::Vector, G::Matrix, h::Vector)
    @constraint(model, A*sol .>= b)
end

function add_inverse_objective!(model::Model, A::Matrix, b::Vector)
end

function create_inverse_problem_single(params::ForwardProblemParams, x_sol::Matrix, z_sol::Vector)
    model = Model(Gurobi.Optimizer)

    A, G, h = create_AGh(params)
    sol = create_solution_vector(x_sol, z_sol)
    println(size(sol))
    G, h = add_half_space_constraint(G, h, sol, params)
    
    b = create_b_variables!(model, params.n_commodities)

    add_inverse_constraints!(model, sol, A, b, G, h)
    add_inverse_objective!(model, A, b)    

    return model
end

create_inverse_problem_single (generic function with 1 method)

In [44]:
params = forward_example_params()

forward_model = create_forward_problem(params)
x_sol, z_sol = solve_forward_problem!(forward_model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-23
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 6 columns and 11 nonzeros
Model fingerprint: 0xa664e8a1
Variable types: 4 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 1e+01]
Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 810 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.100000000000e+02, best bound 8.100000000000e+02, gap 0.0000%

User-callback calls 21

([10.0 0.0; 0.0 6.0], [1.0, 1.0])

In [45]:
inverse_model = create_inverse_problem_single(params, x_sol, z_sol)
print(inverse_model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-23
(6,)
Feasibility


Subject to
 

b[1] + b[3] = 0.0
 b[2] + b[4] = 0.0
 -b[1] ≥ -10.0
 -b[2] ≥ -6.0
 -b[3] ≥ 10.0
 -b[4] ≥ 6.0


In [47]:
optimize!(inverse_model)
value.(inverse_model[:b])

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 4 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 1e+01]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00

User-callback calls 15, time in user-callback 0.00 sec


4-element Vector{Float64}:
  10.0
   6.0
 -10.0
  -6.0